In [178]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model

In [179]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("../general/ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [180]:
demo = pd.read_csv("../PTDEMOG_07Nov2023.csv")

In [181]:
neuro = pd.read_csv("../NEUROEXM_07Nov2023.csv")

In [182]:
neuro.columns

Index(['Phase', 'ID', 'PTID', 'RID', 'SITEID', 'VISCODE', 'VISCODE2',
       'VISDATE', 'USERDATE', 'USERDATE2', 'EXAMDATE', 'NXVISUAL', 'NXAUDITO',
       'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER', 'NXHEEL',
       'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER', 'NXABNORM',
       'update_stamp'],
      dtype='object')

In [183]:
clinical = pd.read_csv("../general/ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})

In [184]:
clinical.head()

,RID,SUBJID,Phase,VISCODE,VISCODE2,EXAMDATE,PHC_VISIT,PHC_Sex,PHC_Education,PHC_Ethnicity,PHC_Race,PHC_AGE,PHC_Diagnosis,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI1,bl,bl,2005-09-08,1,1.0,16.0,2.0,5.0,74.439425,1.0,0.277,0.254,0.293,NaN,2022-10-18 09:15:45.0
1,2,ADNI_011_S_0002,ADNI1,m06,m06,2006-03-06,2,1.0,16.0,2.0,5.0,74.929500,1.0,0.237,0.278,0.535,-0.333,2022-10-18 09:15:45.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,3,1.0,16.0,2.0,5.0,77.407255,1.0,0.344,0.419,0.535,NaN,2022-10-18 09:15:45.0
3,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,4,1.0,16.0,2.0,5.0,79.477070,1.0,0.101,0.066,0.620,0.264,2022-10-18 09:15:45.0
4,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,5,1.0,16.0,2.0,5.0,80.468172,1.0,0.090,0.002,0.460,0.264,2022-10-18 09:15:45.0


In [185]:
diag["Subject"].value_counts()

068_S_2187    3
007_S_2394    3
067_S_2301    3
022_S_2379    3
031_S_2018    2
             ..
305_S_6899    1
020_S_6901    1
036_S_6897    1
137_S_6893    1
305_S_6810    1
Name: Subject, Length: 416, dtype: int64

In [186]:
comb = pd.read_csv("../general/DXSUM_PDXCONV_ADNIALL.csv")[["RID", "PTID" , "Phase"]]

In [187]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [188]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [189]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [190]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [191]:
m["PTID"].value_counts()

067_S_4767    18
141_S_0697    15
067_S_4072    14
067_S_4054    12
130_S_1200    10
              ..
016_S_4963     1
094_S_4459     1
016_S_4575     1
036_S_4740     1
131_S_6153     1
Name: PTID, Length: 109, dtype: int64

In [192]:
t = m

In [193]:
t.columns

Index(['RID', 'PTID', 'Phase', 'ID', 'SITEID', 'VISCODE', 'VISCODE2',
       'VISDATE', 'USERDATE', 'USERDATE2', 'PTSOURCE', 'PTGENDER', 'PTDOB',
       'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTWORKHS', 'PTNOTRT',
       'PTRTYR', 'PTRTYR_DRVD', 'PTHOME', 'PTTLANG', 'PTPLANG', 'PTCOGBEG',
       'PTMCIBEG', 'PTADBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'update_stamp',
       'EXAMDATE', 'NXVISUAL', 'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE',
       'NXMOTOR', 'NXFINGER', 'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA',
       'NXGAIT', 'NXOTHER', 'NXABNORM', 'SUBJID', 'PHC_VISIT', 'PHC_Sex',
       'PHC_Education', 'PHC_Ethnicity', 'PHC_Race', 'PHC_AGE',
       'PHC_Diagnosis', 'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [194]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp",  "PTSOURCE"], axis=1)  #"PTDOBMM", "DX"], axis=1) 
# these variables don't occur in the new DF. Ignorming them for now, but will
# investigate what they are

In [195]:
t.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOB', 'PTDOBYY',
       'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTWORKHS', 'PTNOTRT', 'PTRTYR',
       'PTRTYR_DRVD', 'PTHOME', 'PTTLANG', 'PTPLANG', 'PTCOGBEG', 'PTMCIBEG',
       'PTADBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'EXAMDATE', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'SUBJID', 'PHC_VISIT', 'PHC_Sex', 'PHC_Education',
       'PHC_Ethnicity', 'PHC_Race', 'PHC_AGE', 'PHC_Diagnosis', 'PHC_MEM',
       'PHC_EXF', 'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [196]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

# EVERYHTING UP TO HERE WORKS, THIS IS WHERE IT BEGINS TO BE PROBLEMATIC. I CAN'T
# KEEP WORKING NOW, BUT I WILL COME BACK TO THIS


# This drops PTWORKHS, which seems necessary for the next code block. I'm commenting
# that out for now because it doesn't seem all that relevant for analysis,
# but we should look at why we don't have PTWORK and if these deletions are necessary

In [197]:
len(t.columns)

47

In [198]:
# t["PTWORKHS"] = t["PTWORKHS"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper",
                                                                                                                #    "bookkeeper").str.replace("cpa", "accounting")
# I'm assuming PTWORKHS is what used to be PTWORK
# This part needs to be reviewed

In [199]:
# t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)

#again, dealing with the PTWORK is causing issues

In [200]:
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer') 
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer') 
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

In [201]:
# cond = t['PTWORK'].value_counts()
# threshold = 10
# t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

In [202]:
categorical = ['PTGENDER', #'PTWORK',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [203]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [204]:
text = ["CMMED", 
        # "PTWORK", 
        ]

In [205]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[cols_left]

,PHC_LAN,NXOTHER,PHC_AGE,PHC_Diagnosis,PHC_Education,PHC_VISIT,PHC_EXF,NXTENDON,SUBJID,NXPLANTA,...,PHC_VSP,PTCOGBEG,NXGAIT,PTETHCAT,NXSENSOR,PTRACCAT,PHC_Sex,PTRTYR,NXABNORM,NXNERVE
0,0.364,1.0,62.819986,2.0,12.0,1,0.029,1.0,G-ADNI-AN000478,1.0,...,-0.554,-4.0,1.0,2.0,1.0,5.0,1.0,-4,1.0,1.0
1,0.321,1.0,63.296372,2.0,12.0,2,0.271,1.0,G-ADNI-AN000478,1.0,...,-4.000,-4.0,1.0,2.0,1.0,5.0,1.0,-4,1.0,1.0
2,0.251,1.0,63.794661,3.0,12.0,3,-0.157,1.0,G-ADNI-AN000478,1.0,...,-0.861,-4.0,1.0,2.0,1.0,5.0,1.0,-4,1.0,1.0
3,0.032,1.0,64.292950,2.0,12.0,4,-0.520,1.0,G-ADNI-AN000478,1.0,...,-0.996,-4.0,1.0,2.0,1.0,5.0,1.0,-4,1.0,1.0
4,-0.249,1.0,64.793977,2.0,12.0,5,-0.349,1.0,G-ADNI-AN000478,1.0,...,-0.204,-4.0,1.0,2.0,1.0,5.0,1.0,-4,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,-4.000,1.0,71.638604,-4.0,14.0,1,-4.000,1.0,-4,1.0,...,-4.000,9999.0,1.0,2.0,1.0,4.0,2.0,2015-11-05,1.0,1.0
442,0.820,1.0,62.324435,2.0,16.0,1,1.296,1.0,-4,1.0,...,0.483,2015.0,1.0,2.0,1.0,5.0,2.0,2012-xx-xx,1.0,1.0
443,0.553,-4.0,75.206023,-4.0,16.0,1,0.719,1.0,-4,1.0,...,0.264,2017.0,1.0,2.0,2.0,5.0,1.0,1994-xx-xx,1.0,1.0
444,0.188,1.0,73.234771,3.0,16.0,1,-0.512,1.0,-4,1.0,...,-4.000,2018.0,2.0,2.0,1.0,5.0,1.0,2019-01-xx,1.0,1.0


In [206]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

NXOTHER
PHC_Diagnosis
PHC_Education
PHC_VISIT
NXTENDON
NXPLANTA
NXHEEL
PHC_Race
PTTLANG
PHC_Ethnicity
NXMOTOR
PTADDX
NXCONSCI
NXFINGER
NXGAIT
PTETHCAT
NXSENSOR
PTRACCAT
PHC_Sex
NXABNORM
NXNERVE


In [207]:
t.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOB', 'PTDOBYY',
       'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTRTYR', 'PTRTYR_DRVD',
       'PTHOME', 'PTTLANG', 'PTPLANG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT',
       'PTRACCAT', 'NXVISUAL', 'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE',
       'NXMOTOR', 'NXFINGER', 'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA',
       'NXGAIT', 'NXOTHER', 'NXABNORM', 'SUBJID', 'PHC_VISIT', 'PHC_Sex',
       'PHC_Education', 'PHC_Ethnicity', 'PHC_Race', 'PHC_AGE',
       'PHC_Diagnosis', 'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [208]:
to_del = ["PTRTYR", 
        #   "EXAMDATE", "SUBJECT_KEY", "PTWRECNT"
          ]
t = t.drop(to_del, axis=1)
# removing nonexistent cols

In [209]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[quant]

,SUBJID,PHC_VSP,PHC_LAN,PHC_AGE,PTCOGBEG,PTDOBYY,PTRTYR_DRVD,VISDATE,PTDOB,PHC_MEM,PHC_EXF
0,G-ADNI-AN000478,-0.554,0.364,62.819986,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.509,0.029
1,G-ADNI-AN000478,-4.000,0.321,63.296372,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.248,0.271
2,G-ADNI-AN000478,-0.861,0.251,63.794661,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.626,-0.157
3,G-ADNI-AN000478,-0.996,0.032,64.292950,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.468,-0.520
4,G-ADNI-AN000478,-0.204,-0.249,64.793977,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.616,-0.349
...,...,...,...,...,...,...,...,...,...,...,...
441,-4,-4.000,-4.000,71.638604,9999.0,1948.0,2015.0,2019-03-12,01/1948,0.151,-4.000
442,-4,0.483,0.820,62.324435,2015.0,1957.0,2012.0,2019-06-05,03/1957,0.267,1.296
443,-4,0.264,0.553,75.206023,2017.0,1944.0,1994.0,2019-02-22,10/1944,-0.768,0.719
444,-4,-4.000,0.188,73.234771,2018.0,1946.0,2019.0,2019-10-30,09/1946,-0.386,-0.512


In [210]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [211]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 
       # 'PTWORK', 
       'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [212]:
t[l]

,RID,PTID,Group,Phase,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTNOTRT,...,NXHEEL,NXSENSOR,NXTENDON,NXPLANTA,NXGAIT,NXABNORM,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP
0,429,136_S_0429,1.0,ADNI1,1.0,1943.0,1.0,1.0,12.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.509,0.029,0.364,-0.554
1,429,136_S_0429,1.0,ADNI1,1.0,1943.0,1.0,1.0,12.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.248,0.271,0.321,-4.000
2,429,136_S_0429,1.0,ADNI1,1.0,1943.0,1.0,1.0,12.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.626,-0.157,0.251,-0.861
3,429,136_S_0429,1.0,ADNI1,1.0,1943.0,1.0,1.0,12.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.468,-0.520,0.032,-0.996
4,429,136_S_0429,1.0,ADNI1,1.0,1943.0,1.0,1.0,12.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.616,-0.349,-0.249,-0.204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,6701,032_S_6701,0.0,ADNI3,2.0,1948.0,1.0,1.0,14.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.151,-4.000,-4.000,-4.000
442,6525,067_S_6525,0.0,ADNI3,2.0,1957.0,1.0,1.0,16.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.267,1.296,0.820,0.483
443,6693,137_S_6693,1.0,ADNI3,1.0,1944.0,1.0,1.0,16.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,-0.768,0.719,0.553,0.264
444,6810,305_S_6810,1.0,ADNI3,1.0,1946.0,1.0,1.0,16.0,1.0,...,2.0,1.0,1.0,1.0,2.0,1.0,-0.386,-0.512,0.188,-4.000


In [213]:
dfs = []

In [214]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [215]:
cat = pd.concat(dfs, axis=1)

In [216]:
t[quant]

,SUBJID,PHC_VSP,PHC_LAN,PHC_AGE,PTCOGBEG,PTDOBYY,PTRTYR_DRVD,VISDATE,PTDOB,PHC_MEM,PHC_EXF
0,G-ADNI-AN000478,-0.554,0.364,62.819986,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.509,0.029
1,G-ADNI-AN000478,-4.000,0.321,63.296372,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.248,0.271
2,G-ADNI-AN000478,-0.861,0.251,63.794661,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.626,-0.157
3,G-ADNI-AN000478,-0.996,0.032,64.292950,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.468,-0.520
4,G-ADNI-AN000478,-0.204,-0.249,64.793977,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.616,-0.349
...,...,...,...,...,...,...,...,...,...,...,...
441,-4,-4.000,-4.000,71.638604,9999.0,1948.0,2015.0,2019-03-12,01/1948,0.151,-4.000
442,-4,0.483,0.820,62.324435,2015.0,1957.0,2012.0,2019-06-05,03/1957,0.267,1.296
443,-4,0.264,0.553,75.206023,2017.0,1944.0,1994.0,2019-02-22,10/1944,-0.768,0.719
444,-4,-4.000,0.188,73.234771,2018.0,1946.0,2019.0,2019-10-30,09/1946,-0.386,-0.512


In [217]:
cat

,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_1.0,PTHOME_2.0,PTHOME_3.0,PTHOME_4.0,PTHOME_5.0,PTHOME_6.0,PTHOME_8.0,...,PTRACCAT_4.0,PTRACCAT_5.0,PTRACCAT_6.0,PTRACCAT_7.0,PHC_Sex_1.0,PHC_Sex_2.0,NXABNORM_1.0,NXABNORM_2.0,NXNERVE_1.0,NXNERVE_2.0
0,0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
1,0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
2,0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
3,0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
4,0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,0,0,1,1,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,1,0
442,0,0,1,1,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,1,0
443,0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
444,0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0


In [218]:
t[["PTID","RID", "Phase", "Group"]]

,PTID,RID,Phase,Group
0,136_S_0429,429,ADNI1,1.0
1,136_S_0429,429,ADNI1,1.0
2,136_S_0429,429,ADNI1,1.0
3,136_S_0429,429,ADNI1,1.0
4,136_S_0429,429,ADNI1,1.0
...,...,...,...,...
441,032_S_6701,6701,ADNI3,0.0
442,067_S_6525,6525,ADNI3,0.0
443,137_S_6693,6693,ADNI3,1.0
444,305_S_6810,6810,ADNI3,1.0


In [219]:
c = pd.concat([t[["PTID", "RID", "Phase", "Group"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [220]:
c

,PTID,RID,Phase,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_1.0,PTHOME_2.0,PTHOME_3.0,...,PHC_VSP,PHC_LAN,PHC_AGE,PTCOGBEG,PTDOBYY,PTRTYR_DRVD,VISDATE,PTDOB,PHC_MEM,PHC_EXF
0,136_S_0429,429,ADNI1,1.0,0,1,0,1,0,0,...,-0.554,0.364,62.819986,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.509,0.029
1,136_S_0429,429,ADNI1,1.0,0,1,0,1,0,0,...,-4.000,0.321,63.296372,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.248,0.271
2,136_S_0429,429,ADNI1,1.0,0,1,0,1,0,0,...,-0.861,0.251,63.794661,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.626,-0.157
3,136_S_0429,429,ADNI1,1.0,0,1,0,1,0,0,...,-0.996,0.032,64.292950,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.468,-0.520
4,136_S_0429,429,ADNI1,1.0,0,1,0,1,0,0,...,-0.204,-0.249,64.793977,-4.0,1943.0,-4.0,2006-06-05,09/1943,-0.616,-0.349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,032_S_6701,6701,ADNI3,0.0,0,0,1,1,0,0,...,-4.000,-4.000,71.638604,9999.0,1948.0,2015.0,2019-03-12,01/1948,0.151,-4.000
442,067_S_6525,6525,ADNI3,0.0,0,0,1,1,0,0,...,0.483,0.820,62.324435,2015.0,1957.0,2012.0,2019-06-05,03/1957,0.267,1.296
443,137_S_6693,6693,ADNI3,1.0,0,1,0,1,0,0,...,0.264,0.553,75.206023,2017.0,1944.0,1994.0,2019-02-22,10/1944,-0.768,0.719
444,305_S_6810,6810,ADNI3,1.0,0,1,0,1,0,0,...,-4.000,0.188,73.234771,2018.0,1946.0,2019.0,2019-10-30,09/1946,-0.386,-0.512


In [221]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID', 
                  group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

In [222]:
c.to_csv("clinical.csv")

In [223]:
# This code block checks for duplicate patients and removes them from the 
# dataset. Commenting it out for now, but we should see what the overlap does

# #reading in the overlap test set
# ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

# #removing ids from the overlap test set
# c = c[~c["PTID"].isin(list(ts["PTID"].values))]

In [224]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "Group", "label", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values 
y = c["Group"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [225]:
pd.DataFrame(X_train).to_pickle("X_train_c.pkl")
pd.DataFrame(y_train).to_pickle("y_train_c.pkl")

pd.DataFrame(X_test).to_pickle("X_test_c.pkl")
pd.DataFrame(y_test).to_pickle("y_test_c.pkl")